In [30]:
#Import packages
import geemap
import geopandas
import ee
import os
import rasterio
from rasterio.merge import merge
from rasterio.crs import CRS
from rasterio.transform import from_origin
import glob

In [31]:
Map = geemap.Map(center=[20,85], zoom=4)
Map

Map(center=[20, 85], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [32]:
#Load in aoi
bounding_box = 'C:/Users/Jeremy/OneDrive/GIS/Blender/Ananya Map/Output/bounding_box.shp'
aoi = geemap.shp_to_ee(bounding_box)
aoi_geometry = aoi.geometry()

#Add aoi_geometry bounding box to the map
Map.addLayer(aoi_geometry, {}, 'aoi')

#Centre the map over the aoi
Map.centerObject(aoi_geometry, 11) 
  
#Dates over which to create a median composite. Full dates are biased towards summer due to cloud coverage.
start_date = ee.Date('2019-06-01')
end_date = ee.Date('2019-09-30')

#Set cloud filtering variables
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

In [33]:
def get_s2_sr_cld_col(aoi_geometry, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi_geometry)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi_geometry)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [34]:
#Apply the get_s2_sr_cld_col function to build a collection according to the parameters defined above.
s2_sr_cld_col = get_s2_sr_cld_col(aoi_geometry, start_date, end_date)

In [35]:
#Define a function to add the s2cloudless probability layer and derived cloud mask as bands to an S2 SR image input.
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [36]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [37]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [38]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [39]:
s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .select(['B4', 'B3', 'B2'])  # Select bands 4 (Red), 3 (Green), and 2 (Blue)
                             .median())

In [40]:
#Visualise the composite image
Map.addLayer(s2_sr_median, {'bands':['B4',  'B3',  'B2'], 'min':0, 'max':3000, 'gama':1.1}, 'Sentinel 2 Composite')

In [41]:
#Sample region to test fishnet download
region = aoi_geometry
Map.addLayer(region, {}, 'ROI')
Map.centerObject(region)

In [13]:
#Create a fishnet for parallel downloading
fishnet = geemap.fishnet(region, h_interval=5, v_interval=5, delta=0)

#Style and add the fishnet to the map
style = {'color': 'ffff00ff', 'fillColor': '00000000'}
Map.addLayer(fishnet.style(**style), {}, 'Fishnet')

In [ ]:
out_dir = 'H:\Personal\GIS\Blender\Ananya Map\Assets\S2\Cloudless Composite 100m V2'
geemap.download_ee_image_tiles(
    s2_sr_median, 
    fishnet, 
    out_dir, 
    prefix="sentinel_", 
    crs="EPSG:4326", #Formerly EPSG:3857
    scale=100,
    max_tile_size= 10
)

In [27]:
#Experiment with looping through fishnet for export to google drive
# Loop through each feature in the fishnet
for feature in fishnet.getInfo()["features"]:
    tile = geemap.ee.Feature(feature)

    # Clip the image to the current tile
    clipped_image = s2_sr_median.clip(tile)

    # Create an EE asset ID for exporting
    asset_id = 'Earth Engine Outputs/S2_100m_V2'  # Replace with your own path or naming scheme

    # Export the clipped image to Google Drive
    clipped_image = clipped_image.toFloat()  # Ensure the data type is suitable for export
    export_task = geemap.ee_export_image_to_drive(
        image=clipped_image,
        description="s2_cloudless_mosaic_tile_100m",  # Add a description or naming scheme
        folder=asset_id,  # Replace with your desired folder name or naming scheme
        scale=100, # Replace with your desired scale, set to 100 for coarser resolution
        crs="EPSG:4326", 
        region=tile.geometry(),
        maxPixels=10000000000000
    )

In [42]:
#Experiment with looping through orbit paths for export to google drive
# Loop through each feature in the orbit paths shapefile
orbit_paths = 'H:\Personal\GIS\Blender\Ananya Map\Assets\Vector\orbit_bbox.shp'
orbit_ee = geemap.shp_to_ee(orbit_paths)
#Add aoi_geometry bounding box to the map
Map.addLayer(orbit_ee, {}, 'orbit')

In [43]:
for feature in orbit_ee.getInfo()["features"]:
    tile = geemap.ee.Feature(feature)

    # Clip the image to the current tile
    clipped_image = s2_sr_median.clip(tile)

    # Create an EE asset ID for exporting
    asset_id = 'Earth Engine Outputs/S2_100m_V3'  # Replace with your own path or naming scheme

    # Export the clipped image to Google Drive
    clipped_image = clipped_image.toFloat()  # Ensure the data type is suitable for export
    export_task = geemap.ee_export_image_to_drive(
        image=clipped_image,
        description="s2_cloudless_mosaic_tile_100m",  # Add a description or naming scheme
        folder=asset_id,  # Replace with your desired folder name or naming scheme
        scale=100, # Replace with your desired scale, set to 100 for coarser resolution
        crs="EPSG:4326", 
        region=tile.geometry(),
        maxPixels=10000000000000
    )

In [91]:
#Export mosaiced image to Google Drive
# Create an EE asset ID for exporting
asset_id = 'Earth Engine Outputs - 100m_v2'  # Replace with your own path

# Export the image to Google Drive
s2_sr_median = s2_sr_median.toFloat()  # Ensure the data type is suitable for export
export_task = geemap.ee_export_image_to_drive(
    image=s2_sr_median,
    description="s2_cloudless_mosaic",
    folder=asset_id,  # Replace with your desired folder name
    scale=100,
    crs="EPSG:4326",
    region=aoi_geometry,
    maxPixels=10000000000000
)
# Wait for the export task to complete
#while export_task.status()["state"] in ["READY", "RUNNING"]:
 #   pass



In [ ]:
# Once the export is complete, download the asset image
asset_image = geemap.ee.Image(asset_id)
download_url = asset_image.getDownloadURL({
    'scale': 10,
    'crs': 'EPSG:3857'
})
print("Download the asset image from the following URL:", download_url)

In [60]:
# List all the downloaded Sentinel-2 images
image_files = glob.glob("F:/Personal/GIS/Blender/Ananya Map/Assets/S2/Trial 1/*.tif")

# Read the images into a list of rasterio datasets
datasets = [rasterio.open(image) for image in image_files]

# Merge the images into a single mosaic
mosaic, out_trans = merge(datasets)

# Set the coordinate reference system (CRS) for the output mosaic
out_crs = CRS.from_epsg(3857)

# Set the geotransform parameters for the output mosaic
out_transform = from_origin(out_trans.c, out_trans.f, out_trans.a, out_trans.e)

output_path = "C:/Users/Jeremy/OneDrive/GIS/Blender/Ananya Map/Assets/S2 Test 2/Merged"

# Write the mosaic to a new GeoTIFF file
with rasterio.open(output_path, "w", driver="GTiff", crs=out_crs, transform=out_transform, dtype=mosaic.dtype, count=1) as dest:
    dest.write(mosaic, 1)

# Close all the opened datasets
for dataset in datasets:
    dataset.close()

print(f"Mosaic saved to {output_path}")


TypeError: Integer width and height are required.